# Usage example for `sliceoptim`

This notebook provides an example of use for the `sliceoptim` package.  
We present here how it can be used to optimize slicing parameters for the first layer of a print.  
First we import sliceoptim modules and some utilities:

In [1]:
# main modules
import sliceoptim.samples as sp
import sliceoptim.core as core
# utilities
import numpy as np
import pandas as pd

As we want to optimize our settings for a specific printer / filament pair, we must define these entities.

In [2]:
printer = core.Printer(
    name="test_printer",
    bed_size=[220, 220],
    nozzle_diameter=0.4,
    max_speed=120,
    min_speed=5,
)

filament = core.Filament(
    name="pla_test",
    material="pla",
    extrusion_temp_range=[180, 250],
    bed_temp_range=[25, 80],
    diameter=1.75,
)

In this example, we want to find optimal parameters for the first layer bed and extrusion temperatures, height and speed.  
To do so, we define the parametric search space.

In [3]:
### setup parameters space
space = core.ParametersSpace()

space.add_param(name="first-layer-temperature", low=190, high=220)
space.add_param(name="first-layer-bed-temperature", low=40, high=55)
space.add_param(name="first-layer-speed", low=printer.min_speed, high=printer.max_speed * 0.5)
space.add_param(name="first-layer-height", low=printer.nozzle_diameter * 0.7, high=printer.nozzle_diameter)

Perfect! We need now to create an Experiment by providing a stl file for samples and the parametric space.  
Note we also specify the spacing between samples (in mm) to avoid overlappings and toolpath problems.

In [4]:
### setup experiment
experiment = core.Experiment(
    name="first_layer",
    sample_file="../assets/first_layer.stl",
    is_first_layer=True,
    spacing=5,
    printer=printer,
    filament=filament,
    params_space=space,
    output_file="../examples/example.gcode",
)

Now, we can generate the first batch of testing samples. Each sample will be generated from the stl file and randomized slicing parameters from the provided parametric space.

In [5]:
experiment.max_samples_count

36

The computed maximum number of samples for this experiment is of 36.

In [6]:
experiment.create_new_sample_grid(10)

Generated samples grid is directly stored in the experiment object.

In [7]:
experiment.sample_grid_list[0].samples_list

We are ready to generate corresponding gcode:

In [8]:
experiment.sample_grid_list[0].write_gcode()
# or with the shortcut
experiment.write_gcode_for_last_sample_grid()

Design values (samples parameters), quality and samples costs can be retrived as a pandas dataframe.

In [9]:
experiment.to_dataframe()

,first-layer-temperature,first-layer-bed-temperature,first-layer-speed,first-layer-height,temperature,bed-temperature,print_time,quality,cost,sample_grid_id
0,212,41,20,0.387585,212,41,0.626453,None,None,0.0
1,203,42,26,0.286839,203,42,0.721335,None,None,0.0
2,206,43,48,0.393258,206,43,0.327076,None,None,0.0
3,191,45,39,0.343589,191,45,0.412801,None,None,0.0
4,196,47,5,0.363578,196,47,2.339669,None,None,0.0
5,220,48,34,0.333324,220,48,0.519025,None,None,0.0
6,207,48,17,0.304786,207,48,0.871260,None,None,0.0
7,199,50,42,0.377587,199,50,0.362066,None,None,0.0
8,194,52,59,0.299737,194,52,0.410885,None,None,0.0
9,215,54,12,0.319418,215,54,1.133943,None,None,0.0


Experiment samples can be also set up with the method `from_dataframe`.  
However, to register quality values after a print, you must loop directly over samples and register them and compute costs.

In [10]:
quality_values = [0,1,5,7,3,9,4,9,8,2]
for i, sample in enumerate(experiment.sample_grid_list[0].samples_list):
    sample.quality = quality_values[i] 

In [11]:
experiment.compute_and_update_samples_costs()

In [12]:
experiment.to_dataframe()

,first-layer-temperature,first-layer-bed-temperature,first-layer-speed,first-layer-height,temperature,bed-temperature,print_time,quality,cost,sample_grid_id
0,212,41,20,0.387585,212,41,0.626453,0,10.000000,0.0
1,203,42,26,0.286839,203,42,0.721335,1,8.888889,0.0
2,206,43,48,0.393258,206,43,0.327076,5,4.444444,0.0
3,191,45,39,0.343589,191,45,0.412801,7,3.333333,0.0
4,196,47,5,0.363578,196,47,2.339669,3,6.666667,0.0
5,220,48,34,0.333324,220,48,0.519025,9,1.111111,0.0
6,207,48,17,0.304786,207,48,0.871260,4,5.555556,0.0
7,199,50,42,0.377587,199,50,0.362066,9,0.000000,0.0
8,194,52,59,0.299737,194,52,0.410885,8,2.222222,0.0
9,215,54,12,0.319418,215,54,1.133943,2,7.777778,0.0


After this step, we must register results to optimizer and compute a new samples batch.

In [13]:
experiment.register_costs_to_optimizer()
experiment.create_new_sample_grid(5)

Like for the first samples grid (or batch), we can write new gcode and register quality results.

In [14]:
experiment.write_gcode_for_last_sample_grid()
quality_values = [5,10,15,10,8]
for i, sample in enumerate(experiment.sample_grid_list[-1].samples_list):
    sample.quality = quality_values[i]
experiment.compute_and_update_samples_costs()
experiment.register_costs_to_optimizer()

From there, we already can compute an optimal set of parameters !  
However, it is likely than we did not generate enough samples for a reliable results.  
We may repeat the process until we the optimal cost and uncertainty doesn't improve significatively.  
A validation sample can be printed using computed optimal parameters.

In [15]:
optimal_params, optimal_cost, uncertainty = experiment.estim_best_config()
results  = "optimal parameters : {}, \noptimal cost : {}, \nuncertainty : {}".format(optimal_params, optimal_cost, uncertainty)
print(results)

optimal parameters : {'first-layer-temperature': 209.0, 'first-layer-bed-temperature': 52.0, 'first-layer-speed': 50.0, 'first-layer-height': 0.4}, 
optimal cost : 0.7487857018491852, 
uncertainty : 5.01605561790747


In [16]:
experiment.write_validation_sample(optimal_params, "validation.gcode")